## BLB Games EDA

This notebook will explore the characteristics of the games data, independent of any deck analysis or card information. Quick questions like - how many total drafts are represented? How many are incomplete (don't get to 7 wins or 3 losses).

In [1]:
#Imports
import pandas as pd
from scripts import get_games_data

games_df = get_games_data('blb')

games_df.head(2)

Loading game data for blb from Parquet file...


,draft_id,draft_ti fome,game_time,build_index,match_number,game_number,rank,opp_rank,main_colors,splash_colors,...,"tutored_Ygra, Eater of All","deck_Ygra, Eater of All","sideboard_Ygra, Eater of All","opening_hand_Zoraline, Cosmos Caller","drawn_Zoraline, Cosmos Caller","tutored_Zoraline, Cosmos Caller","deck_Zoraline, Cosmos Caller","sideboard_Zoraline, Cosmos Caller",user_n_games_bucket,user_game_win_rate_bucket
0,deaa4cdcd3e84d8e8b5a0ea34a0f9d79,2024-07-30 22:30:39,2024-07-30 22:58:45,0,1,1,gold,NaN,BG,WR,...,0,1,0,0,0,0,0,0,50,0.54
1,deaa4cdcd3e84d8e8b5a0ea34a0f9d79,2024-07-30 22:30:39,2024-07-30 23:13:12,0,2,1,gold,NaN,BG,WR,...,0,1,0,0,0,0,0,0,50,0.54
